# Vgrid key features

You can try out vgrid by using the cloud-computing platforms below without having to install anything on your computer:

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://vgrid.gishub.vn/lab/index.html?path=notebooks/00_intro.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/master)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD)

## Install vgrid

In [2]:
#%pip install vgrid --upgrade

## DGGS Conversion
### latlon2dggs

In [3]:
from vgrid.conversion.latlon2dggs import latlon2h3
lat = 10.775276
lon = 106.706797 
res = 9
h3_id = latlon2h3(lat, lon,10)
h3_id   

'8a65b56628e7fff'

### dggs2geojson     

In [4]:
from vgrid.conversion.dggs2geojson import h32geojson 
h3_geojson = h32geojson(h3_id)
print(h3_geojson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': (((106.70713615426936, 10.774978441229653), (106.70721514572995, 10.775713374905791), (106.70661718075799, 10.776150587194028), (106.70594022237229, 10.77585286364977), (106.70586123315323, 10.77511792678204), (106.70645920007833, 10.774680716650128), (106.70713615426936, 10.774978441229653)),)}, 'properties': {'h3': '8a65b56628e7fff', 'resolution': 10, 'center_lat': 10.7754157, 'center_lon': 106.7065382, 'avg_edge_len': 81.374, 'cell_area': 17202.984}}]}


## DGGS Visualization
### Using folium
Install folium

In [5]:
%pip install folium

Note: you may need to restart the kernel to use updated packages.


Initialize folium map and visualize the output GeoJSON

In [18]:
import folium

m = folium.Map(tiles='CartoDB positron')
h3_layer = folium.GeoJson(
    h3_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'fillOpacity': 0.3,
        'weight': 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['h3', 'resolution', 'cell_area'],
        aliases=['H3 ID', 'Resolution', 'Area (m²)'],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
).add_to(m)

m.fit_bounds(h3_layer.get_bounds()) 


# Display the map
m

### Using leafmap[maplibre]
Install leafmap[maplibre]

In [7]:
%pip install "leafmap[maplibre]"

Note: you may need to restart the kernel to use updated packages.


Initialize leafmap map and visualize the output GeoJSON         

In [19]:
import leafmap.maplibregl as leafmap
m = leafmap.Map(style="positron")
m.add_geojson(
    h3_geojson,
    layer_type="fill",
    name="H3",
    paint={"fill-color": "blue", "fill-opacity": 0.3},
)
m

Container(children=[Row(children=[Col(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass'…

### csv2dggs
Loading csv file containing DGGS cell Ids

In [9]:
import pandas as pd
csv_path = '../data/point.csv'
csv_file = pd.read_csv(csv_path) # modify the path to the point.csv file containing DGGS cell Ids
csv_file.head()

,id,ward,district,ward_id,dist_id,rhealpix,isea4t,isea3h,ease,s2,olc,geohash,qtm,georef,tilecode,mgrs,maidenhead,quadkey,h3,gars
0,1,Ward 5,District 3,77027151,770,R31260361731,1310231333101032203131021002,"132022628,-1037",L1.165767.02,31752f231,7P28QMCP+M7F,w3gv78yjs,42012323103132022,VGBL41144630,z20x835032y492726,48PXS89,OK30is25,13223011130131231220302,8b65b56638aafff,574JK1917
1,2,Ward 2,District 3,77027157,770,R31260364035,1310231333101002023110233100,"132022623,-1041",L1.165767.02,31752f1f5,7P28QM9J+2M6,w3gv785jq,42012323103123330,VGBL40894605,z20x835020y492739,48PXS89,OK30is14,13223011130133001122110,8b65b5663845fff,574JK1916
2,3,Ward 4,District 3,77027148,770,R31260361377,1310231333101123312323031101,"132022630,-1039",L1.165767.02,31752f23f,7P28QMFM+F5J,w3gv79k17,42012323103101031,VGBL40974642,z20x835024y492720,48PXS89,OK30is15,13223011130131230000213,8b65b566312bfff,574JK1916
3,4,Ward 3,District 3,77027154,770,R31260360881,1310231333101020233133102030,"132022625,-1042",L1.165767.02,31752f21b,7P28QMCH+4QG,w3gv789te,42012323103123313,VGBL40764621,z20x835014y492730,48PXS89,OK30is14,13223011130131222130231,8b65b5663b85fff,574JK1916
4,5,Ward 10,District 3,77027145,770,R31260336867,1310231333123303232131322133,"132022636,-1044",L1.165767.02,31752f283,7P28QMJG+GJ3,w3gv76xmr,42012323103100021,VGBL40594687,z20x835005y492698,48PXS89,OK30is17,13223011130130133121131,8b65b566376efff,574JK1916


Convert CSV to DGGS and visualize the output GeoJSON with folium

In [17]:
import json
from vgrid.conversion.csv2dggs import csv2s2
s2_geojson = csv2s2(csv_path,'s2') # csv2s2(csv_path) by default for 's2' column

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles='CartoDB positron')
s2_layer = folium.GeoJson(
    s2_geojson,
    name='s2',
    overlay=True,
    control=True
)

s2_layer = folium.GeoJson(
    s2_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'fillOpacity': 0.3,
        'weight': 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['s2', 'resolution', 'cell_area'],
        aliases=['S2 Token', 'Resolution', 'Area (m²)'],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
).add_to(m)

m.fit_bounds(s2_layer.get_bounds()) 

# Display the map
m

### geojson2dggs
Loading vector layer in GeoJSON format (Point? Muitipoint, Polyline/ Multipolyline, Polygon/ Multipolygon)

In [ ]:
from vgrid.conversion.geojson2dggs.geojson2rhealpix import geojson2rhealpix
import json

with open('../data/polygon.geojson', 'r') as f:
    geojson_data = json.load(f)

Convert vector layer to DGGS and visualize the output GeoJSON with folium

In [29]:
geojson_rhealpix = geojson2rhealpix(geojson_data, resolution=10, compact=False) 
# Change compact to True to see the compact version of the rHEALPix cells


# Visualize the output GeoJSON
import folium

m = folium.Map(tiles='CartoDB positron')

rhealpix_layer = folium.GeoJson(
    geojson_rhealpix,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'fillOpacity': 0.3,
        'weight': 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['rhealpix', 'resolution', 'cell_area'],
        aliases=['rHEALPix ID', 'Resolution', 'Area (m²)'],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
).add_to(m)

m.fit_bounds(rhealpix_layer.get_bounds()) 

# Display the map
m

Processing features: 100%|██████████| 4/4 [00:00<00:00, 15.12it/s]


### dggscompact
Compact DGGS 

In [37]:
from vgrid.conversion.dggscompact import rhealpixcompact
from vgrid.utils.rhealpixdggs.dggs import RHEALPixDGGS
rhealpix_dggs = RHEALPixDGGS()
rhealpix_compacted = rhealpixcompact(rhealpix_dggs,geojson_rhealpix,rhealpix_id='rhealpix')

# Visualize the output GeoJSON
import folium

m = folium.Map(tiles='CartoDB positron')

rhealpixcompacted_layer = folium.GeoJson(
    rhealpix_compacted,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'black',
        'fillOpacity': 0.3,
        'weight': 1
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['rhealpix', 'resolution', 'cell_area'],
        aliases=['rHEALPix ID', 'Resolution', 'Area (m²)'],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
).add_to(m)

m.fit_bounds(rhealpixcompacted_layer.get_bounds()) 

# Display the map
m

Compacting cells : 100%|██████████| 102/102 [00:00<00:00, 1882.18it/s]
